#Import the needed libraries:

---







In [2]:
# First of all we need to download the needed libraries:
!pip install transformers

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from collections import Counter
import pandas as pd
import numpy as np
import string
import math
import re
import torch

###Data reading :

In [ ]:
data=pd.read_csv('Arabic_dataset.csv')

In [ ]:
# taking a copy from the main dataset:
df = data.copy(deep=True)

In [ ]:
#shape of the data :
df.shape

(5314, 2)

In [ ]:
#displaying the dataset:
df.head()

,Title,Article
0,كيف تعمل نماذج الذكاء الاصطناعي مثل ChatGPT؟ -...,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
1,الذكاء الاصطناعي قد يحرمنا من أهم ميزة في الإن...,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
2,"رئيس مايكروسوفت: ""التزييف العميق"" أكبر المخاوف...",قال رئيس مايكروسوفت (Microsoft) براد سميث، الخ...
3,استخبارات غربية: قراصنة صينيون يتجسسون على منش...,قالت وكالات استخبار غربية وشركة مايكروسوفت إن ...
4,مستخدما تقنية الذكاء الاصطناعي.. محتال في الصي...,استخدم محتال في الصين تقنية الذكاء الاصطناعي ل...


In [ ]:
# Dropping the unwanted columns:
df.drop(columns ="Title", inplace = True)

In [ ]:
df.head()

,Article
0,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
1,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
2,قال رئيس مايكروسوفت (Microsoft) براد سميث، الخ...
3,قالت وكالات استخبار غربية وشركة مايكروسوفت إن ...
4,استخدم محتال في الصين تقنية الذكاء الاصطناعي ل...


In [ ]:
#detecting & removing the duplicates articles :
print('# Duplicate : ',df.duplicated().sum())
df.drop_duplicates(inplace=True)

#checking if there is nan values :
print('# NANS:',df.isna().sum())
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

# Duplicate :  382
# NANS: Article    0
dtype: int64


In [ ]:
#shape of the data after removing the duplicates:
df.shape

(4932, 1)

In [ ]:
# Counting the words in each article:
df["word_count"] = df["Article"].apply(lambda x: len(x.split(" ")))
print(df["word_count"].describe())

count    4932.000000
mean      547.650446
std       482.694394
min        13.000000
25%       295.000000
50%       426.000000
75%       621.250000
max      9938.000000
Name: word_count, dtype: float64


In [ ]:
df["char_count"] = df["Article"].apply(len)
print(df["char_count"] .describe())

count     4932.000000
mean      3156.041768
std       2949.771498
min         70.000000
25%       1601.000000
50%       2364.000000
75%       3650.250000
max      59946.000000
Name: char_count, dtype: float64


In [ ]:
df.head(10)

,Article,word_count,char_count
0,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...,1123,6503
1,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...,1331,7460
2,قال رئيس مايكروسوفت (Microsoft) براد سميث، الخ...,292,1773
3,قالت وكالات استخبار غربية وشركة مايكروسوفت إن ...,580,3669
4,استخدم محتال في الصين تقنية الذكاء الاصطناعي ل...,215,1179
5,شهد العقدان الأخيران من القرن العشرين والأول م...,806,4992
6,أشارت الرابطة الألمانية للبنوك إلى ضرورة توخي ...,282,1680
7,"تعتبر ميزة ""التراجع"" (Undo) من الوظائف المهمة ...",203,1098
8,فرضت الهيئة المعنية بتنظيم خصوصية البيانات بال...,217,1340
9,أعلنت شركة آبل إطلاق الإصدارات الجديدة من برنا...,300,1829


In [ ]:
df.drop(columns =['char_count', 'word_count'], axis = 1)

,Article
0,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
1,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...
2,قال رئيس مايكروسوفت (Microsoft) براد سميث، الخ...
3,قالت وكالات استخبار غربية وشركة مايكروسوفت إن ...
4,استخدم محتال في الصين تقنية الذكاء الاصطناعي ل...
...,...
4927,تمت الكتابة بواسطة: رهف الفلاحات آخر تحديث: ١٣...
4928,تمت الكتابة بواسطة: أسماء عبدالعزيز آخر تحديث:...
4929,تمت الكتابة بواسطة: تسنيم شلبي آخر تحديث: ٠٤:٤...
4930,تمت الكتابة بواسطة: رشا الصوالحة تم التدقيق بو...


## Starting the Pre-Processing Stage \:

---




In [ ]:
# Customized Stop words list :
stop_words =[ "تصفحك", "خدماتنا ", "لموقعنا","كوكيز ","وعلى ","سياسة ","الخصوصية","بواسطه","بواسطة", "الكتابه","الكتابة", "تمت","تحديث",'آخر','جميع' ,'الحقوق',' محفوظة ','©','موضوع',' على','تابع', 'الجزيرة','نت','ولكن', 'هلا', 'لو', 'هذي', 'أكثر', 'لم', 'تين', 'إليك', 'ليستا', 'آه', 'وهو', 'قد', 'فإن', 'لعل', 'مذ', 'اللواتي', 'وإن', 'غير', 'هذه', 'إما', 'أقل', 'ليسوا', 'لستما', 'شتان', 'كليهما', 'التي', 'اللتان', 'هما', 'ليست', 'ممن', 'هاك', 'إي', 'لئن', 'ثم', 'عليه', 'تي', 'ته', 'ذواتا', 'أف', 'تلكما', 'مهما', 'سوف', 'هنا', 'أنتم', 'إنه', 'بهما', 'بهن', 'لهن', 'ليت', 'بعض', 'بعد', 'ذه', 'بلى', 'أيها', 'تلكم', 'عدا', 'منها', 'بكم', 'نحن', 'بس', 'كليكما', 'هيا', 'حاشا', 'هيت', 'إذا', 'اللائي', 'ذا', 'ذواتي', 'بنا', 'فمن', 'لي', 'ليس', 'والذي', 'ذلكن', 'لسنا', 'اللاتي', 'أولئك', 'بي', 'اللتين', 'ذات', 'ريث', 'لن', 'فيها', 'لكن', 'لولا', 'لوما', 'ألا', 'كأي', 'هذا', 'كلتا', 'نعم', 'كي', 'إيه', 'حين', 'كما', 'فإذا', 'ذوا', 'هم', 'بها', 'مه', 'هذان', 'ما', 'ليسا', 'آها', 'ها', 'هاتين', 'بل', 'تلك', 'هاتي', 'ذلك', 'كأن', 'إليكم', 'أنت', 'كذا', 'ذلكم', 'حيثما', 'أنى', 'فلا', 'عن', 'لسن', 'لكنما', 'كل', 'عند', 'سوى', 'في', 'كلا', 'إذ', 'حتى', 'كذلك', 'هي', 'بين', 'إن', 'دون', 'بمن', 'الذين', 'عليك', 'ماذا', 'هاته', 'هناك', 'هنالك', 'وإذا', 'له', 'لا', 'ولا', 'بكما', 'كلما', 'وما', 'ذو', 'فيه', 'كيت', 'اللذان', 'لكي', 'أولاء', 'تينك', 'ثمة', 'مما', 'ذلكما', 'ذين', 'ولو', 'منذ', 'بك', 'كيفما', 'بما', 'لست', 'والذين', 'حيث', 'ذانك', 'عسى', 'إنما', 'لكم', 'أين', 'يا', 'أم', 'لهم', 'بخ', 'فيما', 'اللذين', 'لستم', 'ومن', 'من', 'هن', 'هكذا', 'أي', 'لكما', 'هاتان', 'كأنما', 'أن', 'ذاك', 'عما', 'هو', 'لهما', 'الذي', 'أنتن', 'إذما', 'ذينك', 'هؤلاء', 'أما', 'ذي', 'على', 'هاهنا', 'إذن', 'إليكما', 'حبذا', 'هيهات', 'به', 'مع', 'وإذ', 'لك', 'نحو', 'لاسيما', 'أو', 'أنا', 'آي', 'إليكن', 'لنا', 'أوه', 'أينما', 'خلا', 'فيم', 'إنا', 'هل', 'كيف', 'بكن', 'إلى', 'ذان', 'أنتما', 'عل', 'كم', 'لستن', 'لما', 'بهم', 'منه', 'بيد', 'لكيلا', 'كلاهما', 'متى', 'بماذا', 'إلا', 'هذين', 'لدى', 'لها', 'اللتيا', 'كأين']

In [ ]:
# Detection and Cleansing from the text :
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [ ]:
# Removing the hashtags :
def split_into_sentences(text):
    # Custom sentence splitter based on punctuation marks
    punctuation_marks = ['.', '!', '?']
    sentences = []
    current_sentence = ''

    for char in text:
        current_sentence += char
        if char in punctuation_marks:
            sentences.append(current_sentence.strip())
            current_sentence = ''

    if current_sentence:
        sentences.append(current_sentence.strip())

    return sentences

In [ ]:
def remove_html_symbols(text):
    html_symbols = {
        '&amp;': '&',
        '&lt;': '<',
        '&gt;': '>',
        '&quot;': '"',
        '&apos;': "'"
    }

    result = ""
    i = 0 # index of each char in the words

    while i < len(text):
        if text[i] == '&':
            symbol_found = False

            for symbol in html_symbols:
                if text[i:i+len(symbol)] == symbol:
                    result += html_symbols[symbol]
                    i += len(symbol)
                    symbol_found = True
                    break

            if not symbol_found:
                result += text[i]
                i += 1
        else:
            result += text[i]
            i += 1

    return result


In [ ]:
# removing the cyrillic_letters:
def remove_punctation(text):
    # Define Arabic punctuations
    punctuations = "؟!٪٫٬ـ،؛:\"'(){}[]<>\n"

    # Define Cyrillic letters
    cyrillic_letters = "абвгдежзийклмнопрстуфхцчшщъыьэюя"

    # Remove punctuations and Cyrillic letters from the text
    cleaned_text = ""
    for char in text:
        if char not in punctuations and char not in cyrillic_letters:
            cleaned_text += char

    return cleaned_text

In [ ]:
def remove_tashkeel(text):
    diacritical_marks = [
        '\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650',
        '\u0651', '\u0652'
    ]

    cleaned_text = ""

    for char in text:
        if char not in diacritical_marks:
            cleaned_text += char

    return cleaned_text


In [ ]:
def remove_urls(text):
    filtered_sentences = []
    sentence = ""

    for char in text:
        # Check if the current character is the end of a sentence
        if char == '.':
            # Remove URLs from the current sentence
            filtered_sentence = remove_urls_from_sentence(sentence)

            # Add the filtered sentence to the list
            filtered_sentences.append(filtered_sentence)

            # Reset the sentence variable for the next sentence
            sentence = ""
        else:
            # Append the character to the current sentence
            sentence += char

    return filtered_sentences


def remove_urls_from_sentence(sentence):
    # Remove words containing 'http://' or 'https://'
    words = sentence.split()
    filtered_words = [word for word in words if 'http://' not in word and 'https://' not in word]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence



In [ ]:
#Remove Numbers
def remove_numbers(text):
    # Create an empty string to store the result
    result = ""

    # Iterate over each character in the text
    for char in text:
        # Check if the character is not a numeric digit
        if not char.isdigit():
            # Append the character to the result string
            result += char

    return result

In [ ]:
#Remove Multiple Whitespace
def remove_multiple_whitespace(text):
    # Create an empty string to store the result
    result = ""

    # Flag to keep track of consecutive whitespace
    is_whitespace = False

    # Iterate over each character in the text
    for char in text:
        # Check if the character is a whitespace
        if char.isspace():
            # Check if it is the first whitespace encountered
            if not is_whitespace:
                # Append a single space to the result string
                result += " "
                is_whitespace = True
        else:
            # Append the character to the result string
            result += char
            is_whitespace = False

    return result

In [ ]:
def extract_arabic_sentences(corpus):
    arabic_sentences = ""
    sentence = ""

    for char in corpus:
        if char == '.':
            sentence += char
            arabic_sentences += sentence.strip() + " "
            sentence = ""
        elif is_arabic_letter(char) or char.isspace():
            sentence += char

    if sentence:
        arabic_sentences += sentence.strip() + " "

    return arabic_sentences.strip()


def is_arabic_letter(char):
    arabic_letters = [
        '\u0621', '\u0622', '\u0623', '\u0624', '\u0625', '\u0626', '\u0627',
        '\u0628', '\u0629', '\u062A', '\u062B', '\u062C', '\u062D', '\u062E',
        '\u062F', '\u0630', '\u0631', '\u0632', '\u0633', '\u0634', '\u0635',
        '\u0636', '\u0637', '\u0638', '\u0639', '\u063A', '\u0641', '\u0642',
        '\u0643', '\u0644', '\u0645', '\u0646', '\u0647', '\u0648', '\u0649',
        '\u064A', '\u064B', '\u064C', '\u064D', '\u064E', '\u064F', '\u0650',
        '\u0651', '\u0652', '\u0670', '\u0671'
    ]

    return char in arabic_letters


In [ ]:
# combining all the pre-processing step in one function:
def clean(text):
    text = extract_arabic_sentences(text)
    text = remove_urls(text)
    text = remove_html_symbols(text)
    text = remove_punctation(text)
    text = remove_numbers(text)
    text = remove_stop_words(text)
    text = remove_multiple_whitespace(text)
    text = remove_tashkeel(text)
    return text

In [ ]:
# Applying the pre-processing on the documents:
df["Cleaned_data"] = df["Article"].apply(clean)

In [ ]:
df.head()

,Article,word_count,char_count,Cleaned_data
0,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...,1123,6503,البريد الالكتروني نسيت كلمة المرور كلمة المرور...
1,البريد الالكتروني هل نسيت كلمة المرور ؟\nكلمة ...,1331,7460,البريد الالكتروني نسيت كلمة المرور كلمة المرور...
2,قال رئيس مايكروسوفت (Microsoft) براد سميث، الخ...,292,1773,قال رئيس مايكروسوفت براد سميث الخميس أكبر مخاو...
3,قالت وكالات استخبار غربية وشركة مايكروسوفت إن ...,580,3669,قالت وكالات استخبار غربية وشركة مايكروسوفت مجم...
4,استخدم محتال في الصين تقنية الذكاء الاصطناعي ل...,215,1179,استخدم محتال الصين تقنية الذكاء الاصطناعي لتغي...


In [ ]:
#example from the text at cell 2004 :
df.loc[2004,"Cleaned_data"]

'نتيجة اختفت كيلوغرامات الذهب كان يملكها اختفت بضائع المحال السوقرغم القصف الجوي والمعارك تدور شمال الخرطوم حرص الرجل الذهاب مسكنه الطرف الآخر المدينة منطقة بحري الضفة الأخرى للنيل يقسم العاصمة السودانية للوقوف حدثوتأكد خشي منهيروي عبد الرحمن لفرانس برس نهبوا رأسمالي الحلي الذهبية سرقت يبق شيء مطلقاوبتابع أتمنى تتوقف الحرب نحاول نتدبر أمرنا ونبدأ بداية جديدةولكن توقفت الحرب المستمرة الخامس عشر نيسانإبريل قائد الجيش الفريق أول عبد الفتاح البرهان وقائد قوات الدعم السريع محمد حمدان دقلو اقتصاد السودان أحد أفقر بلدان العالم منكوب عدة عقودبعد عشرين عاما العقوبات الدولية عاد السودان المجتمع الدولي إثر سقوط عمر البشير العامولكن انقلاب العام قام الجنرالان المتحاربان اليوم أدى وقف المساعدات الدولية البالغة ملياري دولار حصلت عليها الخرطوم خلال العامين السابقينوهكذا فقدت الدولة السودانية يوم واحد دخلها وهي قيمة المساعدات الدولية كانت تعاني بالفعل الفساد آثار العقوبات الدولية واستقلال جنوب السودان العام أدى خسارة الخرطوم تقريبا حقول النفطولم تعلن السلطات السودانية موازنة العام ولكنها أشادت بتراجع 

# Finding the similarity of the cleaned data :

---



### Similarity using TF_IDF vectors:

In [ ]:
import re
from collections import Counter
import math

def get_word_counts(document):
    # Divide the text into Arabic words
    words = re.findall(r'\w+', document)
    # Calculate the frequency of each word
    word_counts = Counter(words)
    return word_counts

def get_tfidf_vector(word, corpus):
    # Calculate the thematic frequency (TF) of the given word in each document
    tf_values = []
    for document in corpus:
        word_counts = get_word_counts(document)
        word_count = word_counts.get(word, 0)
        tf = word_count / len(word_counts) if len(word_counts) > 0 else 0
        tf_values.append(tf)

    # Calculate the Inverse Documentation Frequency (IDF) for the given word
    num_documents = len(corpus)
    num_documents_with_word = sum(1 for document in corpus if word in get_word_counts(document))
    idf = math.log(num_documents / (1 + num_documents_with_word))

    # Calculation of TF-IDF values
    tfidf_values = [tf * idf for tf in tf_values]

    return tfidf_values

In [ ]:
# Prompting the words:
corpus = df['Cleaned_data']
word1 = input("Enter word 1: ")
word2 = input("Enter word 2: ")
tfidf_value1 = get_tfidf_vector(word1, corpus)
tfidf_value2 = get_tfidf_vector(word2, corpus)

Enter word 1: الذكاء
Enter word 2: الحاسوب


In [ ]:
# Reshaping the vectors:
x1 = np.array(tfidf_value1).reshape(1, -1)
x2 = np.array(tfidf_value2).reshape(1, -1)
cosine_sim = cosine_similarity(x1,x2)
print(cosine_sim)

[[0.04243516]]


# Finding similarity using word embedding with bert:

---



In [ ]:
# Load the data into a list
corpus = df["Cleaned_data"]

# Load the pre-trained BERT model and tokenizer
model_name = "bert-large-uncased"    #2
#model_name = "aubmindlab/bert-base-arabert"    #1
#model_name = "bert-base-multilingual-cased"
#model_name = "UBC-NLP/ARBERT"
#model_name = "asafaya/bert-base-arabic"
#model_name = "aubmindlab/bert-base-arabertv2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Tokenize the words in the corpus
tokenized_words = [tokenizer.tokenize(word) for word in corpus]
print(tokenized_words[:10])


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (4090 > 512). Running this sequence through the model will result in indexing errors


[['ا', '##ل', '##ب', '##ر', '##ي', '##د', 'ا', '##ل', '##ا', '##ل', '##ك', '##ت', '##ر', '##و', '##ن', '##ي', 'ن', '##س', '##ي', '##ت', 'ك', '##ل', '##م', '##ة', 'ا', '##ل', '##م', '##ر', '##و', '##ر', 'ك', '##ل', '##م', '##ة', 'ا', '##ل', '##م', '##ر', '##و', '##ر', 'ا', '##ظ', '##ه', '##ا', '##ر', 'ت', '##ذ', '##ك', '##ر', '##ن', '##ي', 'ا', '##ل', '##ا', '##س', '##م', 'ا', '##ل', '##ا', '##و', '##ل', 'ا', '##ل', '##ا', '##س', '##م', 'ا', '##ل', '##ا', '##خ', '##ي', '##ر', 'ا', '##ل', '##ب', '##ر', '##ي', '##د', 'ا', '##ل', '##ا', '##ل', '##ك', '##ت', '##ر', '##و', '##ن', '##ي', 'ك', '##ل', '##م', '##ة', 'ا', '##ل', '##م', '##ر', '##و', '##ر', 'ا', '##ظ', '##ه', '##ا', '##ر', 'ف', '##ق', '##د', '##ت', 'ك', '##ل', '##م', '##ة', 'ا', '##ل', '##م', '##ر', '##و', '##ر', 'ا', '##ل', '##خ', '##ا', '##ص', '##ة', 'ي', '##ر', '##ج', '##ى', 'ا', '##د', '##خ', '##ا', '##ل', 'ع', '##ن', '##و', '##ان', 'ب', '##ر', '##ي', '##د', '##ك', 'ا', '##ل', '##ا', '##ل', '##ك', '##ت', '##ر', '##و', '##ن', '

In [ ]:
# Generate word embeddings to the words:
def generate_word_embeddings(words):
    # Convert the words to input tensors
    inputs = tokenizer(words, padding=True, truncation=True, return_tensors="pt")

    # Get the BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Average over the tokens

    # Convert the embeddings to a numpy array
    word_embeddings = embeddings.numpy()

    return word_embeddings

# Prompting the two words
word1 = input("Enter word 1: ")
word2 = input("Enter word 2: ")

# Generate embeddings for the prompted words
word_embeddings = generate_word_embeddings([word1, word2])
word1_embeddings = word_embeddings[0]
word2_embeddings = word_embeddings[1]

# Calculate the similarity between the embeddings
similarity_score = cosine_similarity([word1_embeddings], [word2_embeddings])[0][0]
print(f"Similarity between '{word1}' and '{word2}': {similarity_score}")


Enter word 1: الذكاء
Enter word 2: الحاسوب
Similarity between 'الذكاء' and 'الحاسوب': 0.8850722312927246


# Search Engine part:

---



search for the prompted word in the corpus then return the articles that have the word back with its index.
Then, count the total occurrence of that word:

In [ ]:
# Arabic documents
documents = df['Cleaned_data']

# Function to perform Arabic search
def search(query):
    results = []
    for i, document in enumerate(documents):
        if re.search(query, document, re.UNICODE):
            results.append(i)
    return results

# User input for search query
query = input("Enter Arabic search query: ")

# Applying the function that perform search
results = search(query)

# Display search results
if results:
    print("Search Results:")
    for idx in results:
        print(f"Document {idx + 1}: {documents[idx]}")
else:
    print("No results found.")

# Function that count the occurrence of the word
def search(query):
    count = 0
    for document in documents:
        matches = re.findall(query, document, re.UNICODE)
        count += len(matches)
    return count

# Perform occurence count
count = search(query)

# Display search results
print(f"Total occurrences of '{query}': {count}")

Enter Arabic search query: الحاسوب
Search Results:
Document 10: أعلنت شركة آبل إطلاق الإصدارات الجديدة برنامج تحرير الفيديو فاينال كت برو وبرنامج الموسيقى لوجيك برو لحواسيب آيباد اللوحيةوأوضحت الشركة الأميركية التطبيقات الجديدة تأتي بواجهة مستخدم معدلة تماما وقد انصب اهتمام الشركة تسهيل استعمال التطبيقات المعقدة الحواسيب اللوحيةوأشارت آبل الإصدار الجديد برنامج فاينال كت برو يتضمن ميزة تمرير افتراضية جديدة تسمى جوغ وييل تتيح للمستخدم التنقل المخطط الزمني أجل تحرير الصور بشكل مفرد وبدقة عالية بالإضافة وظيفة لايف دراونغ تسمح بالرسم فوق محتوى الفيديو مباشرة قلم آبلويتيح وضع الكاميرا الاحترافي إمكانية تسجيل مقاطع فيديو مباشرة كاميرا الحاسوب اللوحي بالتنسيق العمودي الأفقي إمكانية التحكم اليدوي درجة الوضوح والتعريض وتوازن اللون الأبيض مباشرةولا تقتصر المزايا الجديدة برنامج الموسيقى لوجيك برو إمكانية مزج التسجيلات الموجودة ومسارات الصوت فحسب يوفر التطبيق الجديد أيضا أدوات برمجية وتأثيرات وآلات إيقاع وآلات توليف يمكن استعمالها طريق اللمسويقوم تطبيق لوجيك برو الجديد بإظهار الأدوات والتصحيحات وال

Search engine with a specified number of articles to display:

In [ ]:
# Arabic documents
documents = df['Cleaned_data']

# Function to search and specify the number of save articles containing the word
def search_and_save(query, num_articles):
    results = []
    count = 0
    for i, document in enumerate(documents):
        if re.search(query, document, re.UNICODE):
            results.append(document)
            count += 1
        if count == num_articles:
            break
    return results

# User input for word and number of articles
query = input("Enter Arabic word: ")
num_articles = int(input("Enter the number of articles to save: "))

# Search and save articles
articles = search_and_save(query, num_articles)

# Display the saved articles
print(f"Saved Articles with '{query}' (up to {num_articles} articles):")
for i, article in enumerate(articles):
    print(f"Article {i + 1}:")
    print(article)
    print("-------------------------")

# Function that count the occurrence of performed search
def search(query):
    count = 0
    for document in documents:
        matches = re.findall(query, document, re.UNICODE)
        count += len(matches)
    return count

# Perform search
count = search(query)

# Display search results
print(f"Total occurrences of '{query}': {count}")


Enter Arabic word: الحاسوب
Enter the number of articles to save: 10
Saved Articles with 'الحاسوب' (up to 10 articles):
Article 1:
أعلنت شركة آبل إطلاق الإصدارات الجديدة برنامج تحرير الفيديو فاينال كت برو وبرنامج الموسيقى لوجيك برو لحواسيب آيباد اللوحيةوأوضحت الشركة الأميركية التطبيقات الجديدة تأتي بواجهة مستخدم معدلة تماما وقد انصب اهتمام الشركة تسهيل استعمال التطبيقات المعقدة الحواسيب اللوحيةوأشارت آبل الإصدار الجديد برنامج فاينال كت برو يتضمن ميزة تمرير افتراضية جديدة تسمى جوغ وييل تتيح للمستخدم التنقل المخطط الزمني أجل تحرير الصور بشكل مفرد وبدقة عالية بالإضافة وظيفة لايف دراونغ تسمح بالرسم فوق محتوى الفيديو مباشرة قلم آبلويتيح وضع الكاميرا الاحترافي إمكانية تسجيل مقاطع فيديو مباشرة كاميرا الحاسوب اللوحي بالتنسيق العمودي الأفقي إمكانية التحكم اليدوي درجة الوضوح والتعريض وتوازن اللون الأبيض مباشرةولا تقتصر المزايا الجديدة برنامج الموسيقى لوجيك برو إمكانية مزج التسجيلات الموجودة ومسارات الصوت فحسب يوفر التطبيق الجديد أيضا أدوات برمجية وتأثيرات وآلات إيقاع وآلات توليف يمكن استعمالها طر